# Winners-Losers


## [Repast for Python (Repast4Py) User Guide](https://repast.github.io/repast4py.site/guide/user_guide.html)

## [API](https://repast.github.io/repast4py.site/apidoc/index.html)

## [GitHub Repast/repast4py](https://github.com/Repast/repast4py)

## [MPI for Python](https://mpi4py.readthedocs.io/en/stable/tutorial.html#collective-communication)

## [A note on Winners-Losers improvements after the first version](aNote.txt)



ESC $\ell$ set or unset row numbers


# An idea for the initial example: 

#### something close to the *Chakraborti model* in "Winners, Losers" ex. in [Is Inequality Inevitable? (Sc.Am.)](https://www.scientificamerican.com/article/is-inequality-inevitable/). In a closer way, § 2.1 in [Chakraborti, A. (2002). Distributions of money in model markets of economy. International Journal of Modern Physics C, 13(10), 1315-1321](https://arxiv.org/pdf/cond-mat/0205221.pdf). 



## run the code as a notebook (single rank) or, opening a terminal, with

## mpirun -n X ipython winners-losers.ipynb

#### where X is the number of ranks

## run plots.ipynb as notebook to show the results

#### plots.ipynb automatically knows the rank number and the root of the name of the result files

===========================================================================

## 1

import libs
MPI init
context and runner definition
t(), T(), Tc(), tr() function definitions
random number generator rng creation
initialization of the parameters from yaml file

===========================================================================


In [1]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import json
import numpy as np
import csv
import os


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() #pt

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initializes the default schedule runner, HERE to create the t() function,
# returning the tick value
"""
init_schedule_runner(comm)
Initializes the default schedule runner, a dynamic schedule of executable 
events shared and synchronized across processes.
Events are added to the scheduled for execution at a particular tick. 
The first valid tick is 0. Events will be executed in tick order, earliest 
before latest. Events scheduled for the same tick will be executed in the 
order in which they were added. If during the execution of a tick, 
an event is scheduled before the executing tick (i.e., scheduled to occur in 
the past) then that event is ignored. The scheduled is synchronized across 
process ranks by determining the global cross-process minimum next scheduled 
event time, and executing only the events schedule for that time. In this way, 
no schedule runs ahead of any other.
"""
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return runner.schedule.tick


# https://repast.github.io/repast4py.site/apidoc/source/repast4py.parameters.html
"""
init_params(parameters_file, parameters)
Initializes the repast4py.parameters.params dictionary with the model input parameters.
"""
params = parameters.init_params("winners-losers.yaml", "")



"""
repast4py.random.default_rng: numpy.random._generator.Generator = Generator(PCG64) 
at 0x7F6812E0CD60 repast4py’s default random generator created using init. 
See the Generator API documentation for more information on the available distributions 
and sampling functions.

Type
numpy.random.Generator

repast4py.random.init(rng_seed=None)
Initializes the default random number generator using the specified seed.

Parameters
rng_seed (int) – the random number seed. Defaults to None in which case, the current 
time as returned by time.time() is used as the seed.
"""

repast4py.random.init(rng_seed=params['myRandom.seed'][rank])
rng = repast4py.random.default_rng 



#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime

T()

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime

Tc()

# count transactions
transactions = 0
def tr(total=False):
    global transactions
    if not total: transactions+=1
    return transactions

cpuTime = [["0 initGhostsIfAny",0],["1 counter",0],["2 agentsChoosingCounterparts",0],\
           ["3 broadcastGhostRequests1",0],["4 request_agents(ghostsToRequest...)1",0],\
           ["5 agentsExchangingInTheirRanks",0],["6 sync1",0],\
           ["7 ghostsExchangingInDifferentRanks",0],\
           ["8 agentsHavingExchangedWithGhostsPreparingTheirOwnGhosts",0],\
           ["9 broadcastGhostRequests2",0],["10 request_agents(ghostsToRequest...)2",0],\
           ["11 sync2",0],["12 messengerGhostsReportingOccuredExchanges",0]]



def losingTime(k):
    for _ in range(k):
        [9,8,7,6,5,4,3,2,1,0].sort()
        
kkk=params['losingTime']

===========================================================================

## 2

memory allocations to manage agents

===========================================================================

In [2]:
agent_cache={} # dict with uid as keys and agents' tuples as values, 
               # used by restore_agent to avoid rebuild agents
    
ghostsToRequest=[] # list of tuples containing for each ghost the uid and its current rank;
                   # used by the requestGhosts(self) function of the model


===========================================================================

## 3

agent classes and restore_agent function

===========================================================================

In [3]:
class WinnerLoser(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, wallet: float, chosenCounterpartRank: int,\
                myGhostCounterpartId: Tuple, materialWalletValueToBeReported: float):
        super().__init__(id=local_id, type=WinnerLoser.TYPE, rank=rank)

        self.myWallet = wallet

        self.chosenCounterpartRank = chosenCounterpartRank
        self.counterpartLocalId = -1
        
        self.havePresenceAsSelfOrGhost = [False] * rankNum
        self.havePresenceAsSelfOrGhost[rank] = True
        
        self.myGhostCounterpartId = myGhostCounterpartId
        
        self.materialWalletValueToBeReported = materialWalletValueToBeReported
        
        self.movAvElements = []

    
    def movAv(self,x):
        
        self.movAvElements.append(x)
        if len(self.movAvElements) > params['movAvElementNum']: self.movAvElements.pop(0)
     
   
    def identifyingLocalCounterparts(self,allWalletsAndOwnersList,counterpartSetSize):
        self.counterpartSet = []
        #print(allWalletsAndOwnersList, counterpartSetSize, flush=True)
        
        for aCounterpart in range(counterpartSetSize):
            if allWalletsAndOwnersList[aCounterpart][0] > self.myWallet:
                print(self.uid[0], allWalletsAndOwnersList[aCounterpart])
    
    
    def choosingRankAndCreatingItsGhostIfAny(self) -> List:
        if params['rank_interaction']: self.chosenCounterpartRank = int(rng.integers(0,rankNum))
        else:                          self.chosenCounterpartRank = rank
        if not self.havePresenceAsSelfOrGhost[self.chosenCounterpartRank]:
            self.havePresenceAsSelfOrGhost[self.chosenCounterpartRank] = True
            return [self.chosenCounterpartRank, ((self.uid[0], self.TYPE, rank), rank)]
        
    def operatingInItsRank(self):  #00000a
        if self.chosenCounterpartRank == rank:
            tmpListOfAgentsInTheSameRank = list(context.agents(agent_type=0))
            ii=0
            for i in range(len(tmpListOfAgentsInTheSameRank)):
                if self.uid == tmpListOfAgentsInTheSameRank[i].uid: ii=i
            tmpListOfAgentsInTheSameRank.pop(ii)
            counterpart=tmpListOfAgentsInTheSameRank[int(rng.integers(0,len(tmpListOfAgentsInTheSameRank)))]
            commonWallet = self.myWallet + counterpart.myWallet
            share=float(rng.random())
            self.myWallet = commonWallet*share
            self.movAv(self.myWallet)
            counterpart.myWallet = commonWallet*(1-share)
            counterpart.movAv(counterpart.myWallet)
            tr()
            
    def operatingInItsRankFast(self,counterpart): #00000b
        # if the counterpart is the agent itself, we reproduce the case of exchanging with
        # an agent having equal wallet
        if self.chosenCounterpartRank == rank:
            commonWallet = self.myWallet + counterpart.myWallet
            share=float(rng.random())
            self.myWallet = commonWallet*share
            self.movAv(self.myWallet)
            counterpart.myWallet = commonWallet*(1-share)
            counterpart.movAv(counterpart.myWallet)
            losingTime(kkk)
            tr()
            
        
    def actingAsGhost(self, materialsReadyToExchange):
        if materialsReadyToExchange == []: return #maybe unuseful
        if self.chosenCounterpartRank==rank: 
                           # the choice of the WL sending the ghost is to op. here
                           # maybe, the WL has also ghosts in other ranks
            materialCounterpart = materialsReadyToExchange.pop(int(rng.integers(0,len(materialsReadyToExchange))))
            commonWallet = self.myWallet + materialCounterpart.myWallet
            share=float(rng.random())
            self.myWallet = commonWallet*share 
                           # the ghost wallet, not relevant
            self.movAv(self.myWallet) #?????
            materialCounterpart.materialWalletValueToBeReported = self.myWallet
                           # the wallet to be reported the WL sending the ghost
                           # in the while, also the movAa() f. will be activated
            materialCounterpart.myWallet = commonWallet*(1-share)
            materialCounterpart.movAv(materialCounterpart.myWallet)
                           # the counterpart wallet
            tr()
            
            materialCounterpart.myGhostCounterpartId = self.uid
            losingTime(kkk)
            #print("@@@@@@@", materialCounterpart.myGhostCounterpartId, materialCounterpart)
    
    #222222
    """
    def actingAsReportingGhost(self, materialsToReportTo):
        if materialsToReportTo == []: return #maybe unuseful
        if self.myGhostCounterpartId == (): return #because it is not a reportingGhost(messenger)
        
        notFound = True
        i = 0
        while notFound: 
            if materialsToReportTo[i].uid == self.myGhostCounterpartId:
                #print("FOUND", rank, t(), materialsToReportTo[i].uid,\
                #      self.myGhostCounterpartId, self.materialWalletValueToBeReported,\
                #      self.myWallet, flush =True)
                notFound = False
                materialsToReportTo[i].myWallet = self.materialWalletValueToBeReported
                materialsToReportTo[i].movAv(self.materialWalletValueToBeReported)
            else: 
                i+=1
                if i == len(materialsToReportTo): return
    """

    def actingAsReportingGhost(self):
        if self.myGhostCounterpartId == (): return #because no messenge to report
        if self.myGhostCounterpartId[2] == rank:
            context.agent(self.myGhostCounterpartId).myWallet = \
                    self.materialWalletValueToBeReported
            context.agent(self.myGhostCounterpartId).movAv(\
                    self.materialWalletValueToBeReported)
                
            
    def sendingMyGhostToReportTheExchange(self) -> List:

        #1111111
        """
        #return [self.uid[2], (self.uid, self.uid[2])]
        #return [self.myGhostCounterpartId[2], (self.myGhostCounterpartId, self.myGhostCounterpartId[2])]
        return [self.myGhostCounterpartId[2], (self.uid, self.uid[2])]
        """
        if not self.havePresenceAsSelfOrGhost[self.myGhostCounterpartId[2]]:
            self.havePresenceAsSelfOrGhost[self.myGhostCounterpartId[2]]=True
            return [self.myGhostCounterpartId[2], (self.uid, self.uid[2])]
               # sending a ghost from myself to the rank from where the counterpart ghost
               # was coming

        
     

    def save(self) -> Tuple: # mandatory, used by request_agents and by synchronize
        #print("save, tick",t(),self.uid[0],self.uid[2],"inRank",rank,flush=True)
        """
        Saves the state of the WinnerLoser as a Tuple.

        Returns:
            The saved state of this WinnerLoser.
        """
        return (self.uid, (self.myWallet, self.chosenCounterpartRank, self.myGhostCounterpartId, \
                           self.materialWalletValueToBeReported))
        # havePresenceAsSelfOrGhost non saved: it is not used in ghost

    
    
    def update(self, dynState: Tuple): # mandatory, used by synchronize
        #print("update, tick",t(),"inRank",rank,flush=True)
        """
        Updates the state of this agent when it is a ghost
        agent on some rank other than its local one.
        """
        self.myWallet=dynState[0]
        self.chosenCounterpartRank = dynState[1]
        self.myGhostCounterpartId = dynState[2]
        self.materialWalletValueToBeReported = dynState[3]
        # havePresenceAsSelfOrGhost not updated: it is not used in ghost
    

      
            
def restore_agent(agent_data: Tuple):
    
    uid=agent_data[0]
    #print("restore, tick",t(),uid,"inRank",rank,flush=True)

    if uid[1] == WinnerLoser.TYPE:
        #print('len agent_cache in restore',len(agent_cache),flush=True)
    
        if uid in agent_cache: 
            tmp = agent_cache[uid] # found
            tmp.myWallet = agent_data[1][0] #restore data
            tmp.chosenCounterpartRank = agent_data[1][1]
            tmp.myGhostCounterpartId = agent_data[1][2]
            tmp.materialWalletValueToBeReported = agent_data[1][3]
            #print("restore1, tick",t(),uid,"inRank",rank,flush=True)


        else: #creation of an instance of the class with its data
            tmp = WinnerLoser(uid[0], uid[2],agent_data[1][0], agent_data[1][1],\
                             agent_data[1][2], agent_data[1][3])                
            agent_cache[uid] = tmp
            #print("restore2, tick",t(),uid,"inRank",rank,flush=True)

        return tmp

    

===========================================================================

## 4

broadcasting function

===========================================================================

In [4]:

def countDigits(n):
    count = 0
    while(n>0):
        count+=1
        n=n//10
    return count


def broadcastGhostRequests(mToBcast, params, rankNum, rank, comm, ghostsToRequest):
    
    n=params['WinnerLoser.count'] // rankNum
    countB = 10+n*(22+countDigits(n))
    if params["sendingGhostsAllAtOnce"]: countB *= rankNum - 1
    str_countB="S"+str(countB)
    
    tmp=-Tc()
    if params["show.timeByTicksAndRanks"]:\
        print("in broadCast - rank",rank,"tick",t(),"len(mToBcast)",len(mToBcast),flush=True)  
    
    mToBcast=json.dumps(mToBcast)
    mToBcast=np.array(mToBcast, dtype=str_countB) 
    mToBcast=mToBcast.tobytes()    
        
    data=[""]*rankNum
    for k in range(rankNum):
        if rank == k:
            data[k] =mToBcast
        else:
            data[k] = bytearray(countB) 
    for k in range(rankNum):
        comm.Bcast(data[k], root=k)

    for k in range(rankNum):
        data[k]=data[k].decode().rstrip('\x00')

    for k in range(rankNum):
        data[k]=json.loads(data[k])
            

    for anItem in data:
        anItem.pop(0)
        for aSubitem in anItem: 
            if len(aSubitem)>1 and aSubitem[0]==rank: 
                aaSubitem = aSubitem[1][0]
                aaSubitem = tuple(aaSubitem)
                aSubitem=(aSubitem[0], (aaSubitem, aSubitem[1][1]))
                    
                if not tuple(aSubitem[1]) in ghostsToRequest:
                    ghostsToRequest.append(tuple(aSubitem[1]))

    tmp+=Tc()
    if params["show.timeByTicksAndRanks"]:\
        print("in broadCast - rank",rank,"tick",t(),"timeWithinBCastFun",tmp,flush=True) 
                    

===========================================================================

## 5

the model

===========================================================================

In [5]:
def returningWalletAndOwner(aWinnerLoser) -> Tuple:
    return (aWinnerLoser.myWallet, aWinnerLoser.uid[0])

class Model:
    """
    The Model class encapsulates the simulation, and is
    responsible for initialization (scheduling events, creating agents,
    and the grid the agents inhabit IF ANY), and the overall iterating
    behavior of the model.

    Args:
        params: the simulation input parameters
    """
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.mToBcast = []
        
        # the context to hold the agents and manage cross process synchronization
        # is created in step 1

        
        # the runner, implementing the schedule, is created in step 1
        # https://repast.github.io/repast4py.site/apidoc/source/repast4py.schedule.html
        
        """
        schedule_repeating_event(at, interval, evt)
        Schedules the specified event to execute at the specified tick, and repeat at 
        the specified interval.

        Parameters
        at (float) – the time of the event.
        interval (float) – the interval at which to repeat event execution.
        evt (Callable) – the Callable to execute when the event occurs.

            A callable is anything that can be called.
            The built-in callable (PyCallable_Check in objects.c) checks if the argument 
            is either:
                an instance of a class with a __call__ method or
                is of a type that has a non null tp_call (c struct) member which 
                indicates callability otherwise (such as in functions, methods etc.)
        """       
        runner.schedule_event(          0.0,     self.initGhosts) #0
        runner.schedule_repeating_event(0.0,  1, self.counter) #1
        #runner.schedule_repeating_event(0.1,  1, self.agentsChoosingCounterpartRank) #2
        runner.schedule_repeating_event(0.1, 1, self.agentsChoosingCounterpartsBothLocallyAndViaGhosts) #2
        runner.schedule_repeating_event(0.11, 1, self.broadcastGhostRequests1) #3    
        runner.schedule_repeating_event(0.12, 1, self.request_agents1) #4    
        #runner.schedule_repeating_event(0.13, 1, self.agentsExchangingInTheirRanks) #5
        runner.schedule_repeating_event(0.13, 1, self.agentsExchangingInTheirRanksFast) #5
        runner.schedule_repeating_event(0.2,  1, self.sync1) #6
        runner.schedule_repeating_event(0.21, 1, self.ghostsExchangingInDifferentRanks) #7
        runner.schedule_repeating_event(0.22, 1,\
                        self.agentsHavingExchangedWithGhostsPreparingTheirOwnGhosts) #8
        runner.schedule_repeating_event(0.23, 1, self.broadcastGhostRequests2) #9
        runner.schedule_repeating_event(0.24, 1, self.request_agents2) #10
        runner.schedule_repeating_event(0.25, 1, self.sync2) #11
        runner.schedule_repeating_event(0.26, 1, 
                        self.messengerGhostsReportingOccuredExchanges) #12
        """
        schedule_stop(at)
        Schedules the execution of this schedule to stop at the specified tick.

        Parameters
        at (float) – the tick at which the schedule will stop.
        """
        runner.schedule_stop(params['howManyCycles'])
        
        runner.schedule_end_event(self.finish)
        

        
        # create agents
        # winnerLoser agents
        
        for i in range(params['WinnerLoser.count'] // rankNum): 
                                                #to subdivide the total #pt
            # create and add the agent to the context
            aWallet= 1 * rng.random() #1
            aWinnerLoser = WinnerLoser(i,rank,aWallet,-1,(), 0)
            context.add(aWinnerLoser)

            

    #initialize ghosts by sending them in the ranks before starting the simulation
    def initGhosts(self):

        cpuTime[0][1]-=Tc()
        if not (params['rank_interaction'] and rankNum>1\
                 and params["sendingGhostsAllAtOnce"]): 
            cpuTime[0][1]+=Tc()
            return

        if params['sendingGhostsAllAtOnce.pushOrPull']=="push": push=True
        else: push=False
        if params['sendingGhostsAllAtOnce.pushOrPull']=="pull": pull=True
        else: pull=False
        
        if params['sendingGhostsAllAtOnce.pushOrPull'] != "push" and \
           params['sendingGhostsAllAtOnce.pushOrPull'] != "pull":
            print("'sendingGhostsAllAtOnce.pushOrPull' has to be 'push' or 'pull'",\
                 flush=True)
            os._exit(os.EX_OK)

        if push:
            #print("1a",Tc(),flush=True)
            self.mToBcast = [rank] 
            rankIds=list(range(rankNum))
            rankIds.pop(rank)
            #print(rank, rankIds, flush=True)
            for aWinnerLoser in context.agents(agent_type=0):
                for rankId in rankIds:
                    self.mToBcast.append([rankId,((aWinnerLoser.uid[0], WinnerLoser.TYPE, rank), rank)])
                    # avoiding successive ghost creation
                    aWinnerLoser.havePresenceAsSelfOrGhost[rankId]=True #operating in this rank
                    
            #print(self.mToBcast, flush = True) 

        if push:
            #print("2",Tc(),flush=True)            
            broadcastGhostRequests(self.mToBcast, Model.PARAMS, rankNum, rank, comm, \
                                    ghostsToRequest)

        if pull:
            #print("1b",Tc(),flush=True)
            #self.mToBcast = [rank] 
            rankIds=list(range(rankNum))
            rankIds.pop(rank)
            #print(rank, rankIds, flush=True)
            for aWinnerLoser in context.agents(agent_type=0):
                for rankId in rankIds:
                    #self.mToBcast.append([rankId,((aWinnerLoser.uid[0], WinnerLoser.TYPE, rank), rank)])
                    # avoiding successive ghost creation
                    aWinnerLoser.havePresenceAsSelfOrGhost[rankId]=True #operating in this rank
            n=params['WinnerLoser.count'] // rankNum
            for rankId in rankIds:
                for i in range(n):
                    ghostsToRequest.append( ((i,WinnerLoser.TYPE,rankId),rankId) )

        #print("3",Tc(),flush=True)
        #print("rank",rank,"ghostsToRequest",ghostsToRequest,flush = True) 
        context.request_agents(ghostsToRequest,restore_agent)
        #print("from initGhosts, rank",rank,"tick",t(),"len(agent_cache)",len(agent_cache),flush=True)
        #print("4",Tc(),flush=True)

        cpuTime[0][1]+=Tc()

                
    
    def counter(self):
        
        cpuTime[1][1]-=Tc()
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
        cpuTime[1][1]+=Tc()
    
    def agentsChoosingCounterpartRank(self):        
        
        cpuTime[2][1]-=Tc()
        del self.mToBcast 
        self.mToBcast = [rank] 
        
        """
        agents(agent_type=None, count=None, shuffle=False)
        Gets the agents in this SharedContext, optionally of the specified type, count 
        or shuffled.

        Parameters
        agent_type (int) – the type id of the agent, defaults to None.
        count (int) – the number of agents to return, defaults to None, meaning return 
        all the agents.shuffle (bool) – whether or not the iteration order is shuffled.
        If true, the order is shuffled. If false, the iteration order is the order of 
        insertion.

        Returns
        An iterable over all the agents in the context. If the agent_type is 
        not None then an iterable over agents of that type will be returned.

        Return type
        iterable 
        pt addendum: it is a generator, not a list
        """
        
        for aWinnerLoser in context.agents(agent_type=0):
            aRequest = aWinnerLoser.choosingRankAndCreatingItsGhostIfAny()
            if aRequest != None: self.mToBcast.append(aRequest)
    
        #print(self.mToBcast, "£££££££££££££££££", rank, t(), flush = True)
        
        cpuTime[2][1]+=Tc()
        
        
        
    def agentsChoosingCounterpartsBothLocallyAndViaGhosts(self):        
        
        cpuTime[2][1]-=Tc()
        del self.mToBcast 
        self.mToBcast = [rank] 
        
        counterpartSetSize=int(params['WinnerLoser.count']*params['counterpartsRatio']/rankNum)
        allWalletsAndOwnersList=[]
        for aWinnerLoser in context.agents(agent_type=0):
            allWalletsAndOwnersList.append(returningWalletAndOwner(aWinnerLoser))
        allWalletsAndOwnersList=sorted(allWalletsAndOwnersList, reverse = True)  
        print(allWalletsAndOwnersList)
        for aWinnerLoser in context.agents(agent_type=0):
            aWinnerLoser.identifyingLocalCounterparts(allWalletsAndOwnersList,counterpartSetSize)
        
        cpuTime[2][1]+=Tc()
        
        
    def broadcastGhostRequests1(self):
        
        cpuTime[3][1]-=Tc()        
        if (not params['rank_interaction']) or rankNum==1 \
                                            or params['sendingGhostsAllAtOnce']: 
            cpuTime[3][1]+=Tc()
            return     

        #if t()<1:
            #print(rank, self.mToBcast, flush=True)
        broadcastGhostRequests(self.mToBcast, Model.PARAMS, rankNum, rank, comm, \
                               ghostsToRequest)  #broadcasting
        
        cpuTime[3][1]+=Tc()        

    def broadcastGhostRequests2(self):
        
        cpuTime[9][1]-=Tc()        
        if (not params['rank_interaction']) or rankNum==1 \
                                            or params['sendingGhostsAllAtOnce']: 
            cpuTime[9][1]+=Tc()
            return     

        broadcastGhostRequests(self.mToBcast, Model.PARAMS, rankNum, rank, comm, \
                               ghostsToRequest)  #broadcasting
        
        cpuTime[9][1]+=Tc()        

    def request_agents1(self):
        
        cpuTime[4][1]-=Tc()        
        if (not params['rank_interaction']) or rankNum==1 \
                                            or params['sendingGhostsAllAtOnce']: 
            cpuTime[4][1]+=Tc()
            return 
        
        """
        https://repast.github.io/repast4py.site/apidoc/source/repast4py.context.html
        request_agents(requested_agents, create_agent)
        Requests agents from other ranks to be copied to this rank as ghosts.

        !!!! This is a collective operation and all ranks must call it, regardless 
        of whether agents are being requested by that rank. The requested agents 
        will be automatically added as ghosts to this rank.

        Parameters
        requested_agents (List) – A list of tuples specifying requested 
        agents and the rank to request from. Each tuple must contain the agents 
        unique id tuple and the rank, for example ((id, type, rank), requested_rank).

        create_agent (Callable) – a Callable that can take the result of an agent 
        save() and return an agent.

        Returns
        ***The list of requested agents.

        Return type
        List[_core.Agent]
        """

        context.request_agents(ghostsToRequest,restore_agent)

        cpuTime[4][1]+=Tc()        

        
    def request_agents2(self):
        
        cpuTime[10][1]-=Tc()        
        if (not params['rank_interaction']) or rankNum==1 \
                                            or params['sendingGhostsAllAtOnce']: 
            cpuTime[10][1]+=Tc()
            return 
        
        """
        https://repast.github.io/repast4py.site/apidoc/source/repast4py.context.html
        request_agents(requested_agents, create_agent)
        Requests agents from other ranks to be copied to this rank as ghosts.

        !!!! This is a collective operation and all ranks must call it, regardless 
        of whether agents are being requested by that rank. The requested agents 
        will be automatically added as ghosts to this rank.

        Parameters
        requested_agents (List) – A list of tuples specifying requested 
        agents and the rank to request from. Each tuple must contain the agents 
        unique id tuple and the rank, for example ((id, type, rank), requested_rank).

        create_agent (Callable) – a Callable that can take the result of an agent 
        save() and return an agent.

        Returns
        ***The list of requested agents.

        Return type
        List[_core.Agent]
        """

        context.request_agents(ghostsToRequest,restore_agent)

        cpuTime[10][1]+=Tc()        

        
    def agentsExchangingInTheirRanks(self): #0000a
        
        cpuTime[5][1]-=Tc()        
        for aWinnerLoser in context.agents(agent_type=0):
            aWinnerLoser.operatingInItsRank()

        cpuTime[5][1]+=Tc()        

 
    def agentsExchangingInTheirRanksFast(self):  #0000b
        
        cpuTime[5][1]-=Tc()
        tmpListOfAgentsInTheSameRank = list(context.agents(agent_type=0,shuffle=True))
        i=0
        for aWinnerLoser in context.agents(agent_type=0):
            aWinnerLoser.operatingInItsRankFast(tmpListOfAgentsInTheSameRank[i])
            i+=1

        cpuTime[5][1]+=Tc()        

   
    
    def ghostsExchangingInDifferentRanks(self):
        cpuTime[7][1]-=Tc()
        if (not params['rank_interaction']) or rankNum==1: 
            cpuTime[7][1]+=Tc()
            return     
        # clean preios initilizations in materials and ghosts
        for aWinnerLoser in context.agents(agent_type=0):
            aWinnerLoser.myGhostCounterpartId = ()
        if not agent_cache == {}:
            currentGhostList=list(agent_cache.keys())
            for i in range(len(agent_cache)):                
                agent_cache[currentGhostList[i]].myGhostCounterpartId = ()
       
        del self.mToBcast 
        self.mToBcast = [rank] 
        
        materialsReadyToExchange = list(context.agents(agent_type=0)).copy()     
        if not agent_cache == {}:
            currentGhostList=list(agent_cache.keys())
            for i in range(len(agent_cache)):                
                agent_cache[currentGhostList[i]].actingAsGhost(materialsReadyToExchange)

        cpuTime[7][1]+=Tc()
       
    
    #preparing mToBcast
    def agentsHavingExchangedWithGhostsPreparingTheirOwnGhosts(self):
        
        cpuTime[8][1]-=Tc()
        if (not params['rank_interaction']) or rankNum==1: 
            cpuTime[8][1]+=Tc()
            return
        for aWinnerLoser in context.agents(agent_type=0):
            if aWinnerLoser.myGhostCounterpartId != ():
                aRequest = aWinnerLoser.sendingMyGhostToReportTheExchange()
                if aRequest != None: self.mToBcast.append(aRequest)
        #print(self.mToBcast, "$$$$$$$$$$$$$$$$$", rank, t(), flush = True)
        
        cpuTime[8][1]+=Tc()
        
    
    def messengerGhostsReportingOccuredExchanges(self):
        cpuTime[12][1]-=Tc()
        if (not params['rank_interaction']) or rankNum==1: 
            cpuTime[12][1]+=Tc()
            return
        
        """
        materialsToReportTo = list(context.agents(agent_type=0)).copy()     
        if not agent_cache == {}:
            currentReportingGhostList=list(agent_cache.keys())
            for i in range(len(agent_cache)):                
                agent_cache[currentReportingGhostList[i]].\
                actingAsReportingGhost(materialsToReportTo)
        """
        #222222 
        if not agent_cache == {}:
            currentReportingGhostList=list(agent_cache.keys())
            for i in range(len(agent_cache)):
                agent_cache[currentReportingGhostList[i]].\
                    actingAsReportingGhost()
        
        cpuTime[12][1]+=Tc()
    

                    
        
    def sync1(self):

        cpuTime[6][1]-=Tc() 
        if (not params['rank_interaction']) or rankNum==1: 
            cpuTime[6][1]+=Tc()
            return             
        
        tmp=-Tc()
        if params["show.timeByTicksAndRanks"]:\
            print("from sync1 - rank",rank,"tick",t(),"len(agent_cache)",len(agent_cache),flush=True)  

       
        """
        synchronize(restore_agent, sync_ghosts=True)
        Synchronizes the model state across processes by moving agents, 
        filling projection buffers with ghosts, updating ghosted state and so forth.

        Parameters
        restore_agent (Callable) – a calluable that takes agent state data and returns 
        an agent instance from that data. The data is a tuple whose first element 
        is the agent’s unique id tuple, and the second element is the agent’s state, 
        as returned by that agent’s type’s save() method.

        sync_ghosts (bool) – if True, the ghosts in any SharedProjections and 
        value layers associated with this SharedContext are also synchronized. 
        Defaults to True.
        """
        
        context.synchronize(restore_agent)
            
        cpuTime[6][1]+=Tc()   

        tmp+=Tc()
        if params["show.timeByTicksAndRanks"]:\
            print("from sync1 - rank",rank,"tick",t(),"timeWithinBCastFun",tmp,flush=True)  
    
    def sync2(self):
        
        cpuTime[11][1]-=Tc() 
        if (not params['rank_interaction']) or rankNum==1: 
            cpuTime[11][1]+=Tc()
            return             
        
        tmp=-Tc()
        if params["show.timeByTicksAndRanks"]:\
            print("from sync2 - rank",rank,"tick",t(),"len(agent_cache)",len(agent_cache),flush=True)  

        """
        synchronize(restore_agent, sync_ghosts=True)
        Synchronizes the model state across processes by moving agents, 
        filling projection buffers with ghosts, updating ghosted state and so forth.

        Parameters
        restore_agent (Callable) – a calluable that takes agent state data and returns 
        an agent instance from that data. The data is a tuple whose first element 
        is the agent’s unique id tuple, and the second element is the agent’s state, 
        as returned by that agent’s type’s save() method.

        sync_ghosts (bool) – if True, the ghosts in any SharedProjections and 
        value layers associated with this SharedContext are also synchronized. 
        Defaults to True.
        """

        context.synchronize(restore_agent)

        cpuTime[11][1]+=Tc()        

        tmp+=Tc()
        if params["show.timeByTicksAndRanks"]:\
            print("from sync2 - rank",rank,"tick",t(),"timeWithinBCastFun",tmp,flush=True)  

                        
    def finish(self):
        allTheWallets = []
        for aWinnerLoser in context.agents(agent_type=0):
            allTheWallets.append(aWinnerLoser.myWallet)
        
        with open(params["log_file_root"]+str(rank)+'.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(allTheWallets)
        
        allTheMovAv = []
        for aWinnerLoser in context.agents(agent_type=0):
            #if aWinnerLoser.uid == (0,0,0): print("%%%%%%",aWinnerLoser.movAvElements)
            if aWinnerLoser.movAvElements != []: allTheMovAv.\
               append(np.sum(aWinnerLoser.movAvElements)/len(aWinnerLoser.movAvElements))
                
            
        print("\n\nBye bye by rank",rank,"at tick",t(),"elapsed time",'%6.3f' % T(),\
              "CPU time",'%6.3f' % Tc(),"transaction #", tr(True), flush=True)
        
        with open(params["log_file_root"]+"MovAv"+str(rank)+'.csv', 'w', newline='') \
          as file:
            writer = csv.writer(file)
            writer.writerow(allTheMovAv)
            
        if params["show.timeBySteps"]:
            print("Intervals in rank ",rank,flush=True)
            for i in range(len(cpuTime)):
                print(cpuTime[i][0],'%6.3f' % cpuTime[i][1],flush=True)
        


    def start(self):

        runner.execute()
        

===========================================================================

## 6

run the model

===========================================================================

In [6]:
# infos for plots.ipynm
with open('plotInfo.csv', 'w', newline='')\
          as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum))

def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

rank 0 tick 0.0
[(0.9406234519742933, 32), (0.9371277593532278, 0), (0.8856282620266805, 2), (0.8844623429064837, 1), (0.8697232031720833, 11), (0.85358984370939, 21), (0.8312688620952662, 8), (0.8299410226815815, 22), (0.7884645201543802, 25), (0.7741105619563224, 14), (0.754360744210118, 4), (0.7395415251846907, 13), (0.730450441467793, 12), (0.7188893752146979, 10), (0.659910984801379, 7), (0.6490014455544135, 15), (0.6425764696083817, 26), (0.6146396268828666, 29), (0.579339616365606, 18), (0.5582317456419214, 23), (0.5231101479968824, 16), (0.4259001706930141, 6), (0.4231172301004603, 24), (0.41824928188001387, 31), (0.3371822234151688, 9), (0.3270825211810946, 5), (0.2968560587419863, 20), (0.2929284019355367, 27), (0.240821974017697, 30), (0.1713953970773301, 19), (0.14714456945205345, 17), (0.06981076426667698, 3), (0.05429496219236629, 28)]
0 (0.9406234519742933, 32)
1 (0.9406234519742933, 32)
1 (0.9371277593532278, 0)
1 (0.8856282620266805, 2)
2 (0.9406234519742933, 32)
2 (0.

13 (0.8697232031720833, 11)
13 (0.85358984370939, 21)
13 (0.8312688620952662, 8)
13 (0.8299410226815815, 22)
13 (0.7884645201543802, 25)
13 (0.7741105619563224, 14)
13 (0.754360744210118, 4)
14 (0.9406234519742933, 32)
14 (0.9371277593532278, 0)
14 (0.8856282620266805, 2)
14 (0.8844623429064837, 1)
14 (0.8697232031720833, 11)
14 (0.85358984370939, 21)
14 (0.8312688620952662, 8)
14 (0.8299410226815815, 22)
14 (0.7884645201543802, 25)
15 (0.9406234519742933, 32)
15 (0.9371277593532278, 0)
15 (0.8856282620266805, 2)
15 (0.8844623429064837, 1)
15 (0.8697232031720833, 11)
15 (0.85358984370939, 21)
15 (0.8312688620952662, 8)
15 (0.8299410226815815, 22)
15 (0.7884645201543802, 25)
15 (0.7741105619563224, 14)
15 (0.754360744210118, 4)
15 (0.7395415251846907, 13)
15 (0.730450441467793, 12)
16 (0.9406234519742933, 32)
16 (0.9371277593532278, 0)
16 (0.8856282620266805, 2)
16 (0.8844623429064837, 1)
16 (0.8697232031720833, 11)
16 (0.85358984370939, 21)
16 (0.8312688620952662, 8)
16 (0.829941022681

### run plots.ipynb to show the results

#### plots.ipynb automatically knows the rank number and the root of the name of the result files